# Patronizing and Condescending Language Detection

In [ ]:
!pip list

In [ ]:
#dependencies
# ! pip install nltk
# ! pip install Unidecode
# ! pip install wordcloud
! python -m pip install nltk
! python -m pip install wordcloud
! python -m pip install Unidecode
! python -m pip install beautifulsoup4

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from dont_patronize_me import DontPatronizeMe
import random
from wordcloud import WordCloud
from transformers import BertTokenizer
from transformers import BertPreTrainedModel, BertModel
from transformers import Trainer, TrainingArguments
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import os
from collections import Counter
from urllib import request
from DPM_preprocessing_over_sampling2_train_test_80_20 import DPM_preprocessing

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# check gpu
# check which gpu we're using
!nvidia-smi
cuda_available = torch.cuda.is_available()

if cuda_available:
    device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print('Cuda available? ',cuda_available)

In [ ]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

# 1.1 Data analysis of the training data

In [31]:
dpm = DontPatronizeMe('.', 'task4_test.tsv')

dpm_pp = DPM_preprocessing('.', '.')

In [ ]:
def count_words(sentence):
    return len(sentence.split())
dpm.load_task1()
dpm.load_task2(return_one_hot=True)
df = dpm.train_task1_df
df['lenght'] = df['text'].apply(count_words)
hist = df['lenght'].hist(by=df['label'], bins = 50, alpha = 0.5)
#hist = df['lenght'].hist(by=df['orig_label'], bins = 10, alpha = 0.5)
#hist = df['country'].hist(by=df['label'])
#hist = df['keyword'].hist(by=df['orig_label'])
plt.savefig('histo.jpg', dpi=500)

print(df.shape)
#Preprocessing and over-sampling
dpm_pp.load_task1()
dpm_pp.load_task2(return_one_hot=True)
dpm_pp.load_all_positive()
dpm_pp.load_all_negative()
df_80_pos = dpm_pp._80_positive_df
df_20_pos = dpm_pp._20_positive_df
df_80_neg = dpm_pp._80_negative_df
df_20_neg = dpm_pp._20_negative_df
df['lenght'] = df['text'].apply(count_words)
hist = df['lenght'].hist(by=df['label'], bins = 50, alpha = 0.5)
#hist = df['lenght'].hist(by=df['orig_label'], bins = 10, alpha = 0.5)
#hist = df['country'].hist(by=df['label'])
#hist = df['keyword'].hist(by=df['orig_label'])
plt.savefig('histo.jpg', dpi=500)


In [ ]:
print(df_80_pos.shape)
print(df_20_pos.shape)
print(df_80_neg.shape)
print(df_20_neg.shape)
train_df = pd.concat([df_80_pos, df_80_neg])
val_df = pd.concat([df_20_pos, df_20_neg])
total_df = pd.concat([train_df, val_df])
print(train_df.shape)
print(val_df.shape)
print(total_df.shape)

In [ ]:
def generate_cloud(label,label_type,df):

    if label_type == "label ":
        text = df[df['label'] == label]['text'].values
    else:
        text = df[df['orig_label'] == label]['text'].values
    
    wordcloud = WordCloud().generate(str(text))
    
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
generate_cloud(0,"label ",dpm.train_task1_df)

In [ ]:
generate_cloud(1,"label ",dpm.train_task1_df)

# 1.2 Qualitative assessment of the dataset

In [ ]:
# To be done

# 2 Modelling

## 2.1 Load training, validation and test set

In [ ]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.head()

In [ ]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)
print("Original Training Length: ",len(trids))
print("Original Test Length: ",len(teids))

In [15]:
train_rows = [] # will contain par_id, label and text
val_rows = []
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label

  train_row = train_df[train_df.par_id==parid]
  for index, row in train_row.iterrows():
    text = train_row.text.values[0]
    label = train_row.label.values[0]
    train_rows.append({
      'par_id':parid,
      'text':text,
      'label':label
    })
#   print(train_row)

  val_row = val_df[val_df.par_id==parid]
  for index, row in val_row.iterrows():
    text = val_row.text.values[0]
    label = val_row.label.values[0]
    val_rows.append({
      'par_id':parid,
      'text':text,
      'label':label
    })
#   print(val_row)

    
trdf1 = pd.DataFrame(train_rows)
valdf1 = pd.DataFrame(val_rows)
# from sklearn.model_selection import train_test_split
# trdf1, valdf1 = train_test_split(trdf1, test_size=0.1)

In [16]:
#over-sampling
print(trdf1.shape)
print(valdf1.shape)

(11614, 3)
(2393, 3)


In [17]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label
  })

tedf1 = pd.DataFrame(rows)

In [18]:
# downsample negative instances
# Training
# pcldf = trdf1[trdf1.label==1]
# npos = len(pcldf)
# training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])
training_set1 = trdf1
# Validation
# pcldf = valdf1[valdf1.label==1]
# npos = len(pcldf)
# validation_set1 = pd.concat([pcldf,valdf1[valdf1.label==0][:npos*2]])
validation_set1 = valdf1
# Testing
# pcldf = tedf1[tedf1.label==1]
# npos = len(pcldf)
# test_set1 = pd.concat([pcldf,tedf1[tedf1.label==0][:npos*2]])
test_set1 = tedf1
print("Training set length: ",len(training_set1))
print("Validation set length: ",len(validation_set1))
print("Testing set length: ",len(test_set1))
print(training_set1.shape)
training_set1

Training set length:  11614
Validation set length:  2393
Testing set length:  2094
(11614, 3)


,par_id,text,label
0,4341,scheme saw an estimated 150000 children from ...,1
1,4341,scheme saw an estimated 150000 children from ...,1
2,4341,scheme saw an estimated 150000 children from ...,1
3,4341,scheme saw an estimated 150000 children from ...,1
4,4341,scheme saw an estimated 150000 children from ...,1
...,...,...,...
11609,7575,are the poor who live in makeshift structures ...,0
11610,7576,on the other hand the argument has been made t...,0
11611,7577,from ahmed zayed in tripoli libya tripoli caj ...,0
11612,7578,the duo was called by their national football ...,0


In [19]:
class DpmDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, input_set):

        self.tokenizer = tokenizer
        self.texts = list(input_set['text'])
        self.labels = list(input_set['label'])
        
    def collate_fn(self, batch):

        texts = []
        labels = []

        for b in batch:
            texts.append(b['text'])
            labels.append(b['label'])

        #The maximum sequence size for BERT is 512 but here the tokenizer truncate sentences longer than 128 tokens.  
        # We also pad shorter sentences to a length of 128 tokens
        encodings = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=256)
        encodings['label'] =  torch.tensor(labels)
        
        return encodings
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
       
        item = {'text': self.texts[idx],
                'label': self.labels[idx]}
        return item

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [21]:
train_dataset = DpmDataset(tokenizer, training_set1)

In [22]:
# Print some example in training set
batch = [sample for sample in train_dataset]

encodings = train_dataset.collate_fn(batch[:10])

for key, value in encodings.items():
  print(f"{key}: {value.numpy().tolist()}")

input_ids: [[101, 5471, 1486, 1126, 3555, 10204, 7629, 1482, 1121, 2869, 2073, 1217, 1850, 1106, 2192, 1104, 1103, 9304, 10721, 1324, 8207, 1206, 3598, 1105, 2424, 1118, 2689, 3791, 1105, 18844, 1150, 1163, 1152, 1156, 1730, 1618, 2491, 102, 0, 0, 0, 0, 0, 0], [101, 5471, 1486, 1126, 3555, 10204, 7629, 1482, 1121, 2869, 2073, 1217, 1850, 1106, 2192, 1104, 1103, 9304, 10721, 1324, 8207, 1206, 3598, 1105, 2424, 1118, 2689, 3791, 1105, 18844, 1150, 1163, 1152, 1156, 1730, 1618, 2491, 102, 0, 0, 0, 0, 0, 0], [101, 5471, 1486, 1126, 3555, 10204, 7629, 1482, 1121, 2869, 2073, 1217, 1850, 1106, 2192, 1104, 1103, 9304, 10721, 1324, 8207, 1206, 3598, 1105, 2424, 1118, 2689, 3791, 1105, 18844, 1150, 1163, 1152, 1156, 1730, 1618, 2491, 102, 0, 0, 0, 0, 0, 0], [101, 5471, 1486, 1126, 3555, 10204, 7629, 1482, 1121, 2869, 2073, 1217, 1850, 1106, 2192, 1104, 1103, 9304, 10721, 1324, 8207, 1206, 3598, 1105, 2424, 1118, 2689, 3791, 1105, 18844, 1150, 1163, 1152, 1156, 1730, 1618, 2491, 102, 0, 0, 0, 0,

## 2.2 Construct Model

In [23]:
class pcl_detection(BertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)

        # BERT Model
        self.bert = BertModel(config)
        
        # Task A
        self.projection_a = torch.nn.Sequential(torch.nn.Dropout(0.2),
                                                torch.nn.Linear(config.hidden_size, 2))
        
        # Task B
        # TBA
        
        # Task C
        # TBA
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
 
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # Logits A
        logits_a = self.projection_a(outputs[1])
        
        return logits_a

In [24]:
# Define Loss function
class Trainer_pcl_detection(Trainer):
    def compute_loss(self, model, inputs):
        labels = {}
        labels['label'] = inputs.pop('label')

        outputs = model(**inputs)

        loss_task_a = nn.CrossEntropyLoss()
        labels = labels['label']
        loss = loss_task_a(outputs.view(-1, 2), labels.view(-1))
        
        return loss

## 2.3 Train

In [25]:
def main():
    
    #model = BERT_hate_speech.from_pretrained(model_name)\
    model = pcl_detection.from_pretrained("bert-base-cased")
    #call our custom BERT model and pass as parameter the name of an available pretrained model    
    training_args = TrainingArguments(
        output_dir='./experiment/hate_speech',
        learning_rate = 0.0001,
        logging_steps= 100,
        per_device_train_batch_size=32,
        num_train_epochs = 3,
    )
    trainer = Trainer_pcl_detection(
        model=model,                         
        args=training_args,                 
        train_dataset=train_dataset,                   
        data_collator=train_dataset.collate_fn
    )
    trainer.train()

    trainer.save_model('./models/pcl_bert_finetuned/')



In [26]:
main()

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing pcl_detection: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing pcl_detection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing pcl_detection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of pcl_detection were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['projection_a.1.weigh

Step,Training Loss
100,0.450300
200,0.221900
300,0.099200
400,0.069700
500,0.022900
600,0.026100
700,0.027400
800,0.004800
900,0.001000
1000,0.000500


Saving model checkpoint to ./experiment/hate_speech/checkpoint-500
Configuration saved in ./experiment/hate_speech/checkpoint-500/config.json
Model weights saved in ./experiment/hate_speech/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./experiment/hate_speech/checkpoint-1000
Configuration saved in ./experiment/hate_speech/checkpoint-1000/config.json
Model weights saved in ./experiment/hate_speech/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./models/pcl_bert_finetuned/
Configuration saved in ./models/pcl_bert_finetuned/config.json
Model weights saved in ./models/pcl_bert_finetuned/pytorch_model.bin


## 2.4 Validation

In [27]:
def predict_pcl(input, tokenizer, model): 
  model.eval()
  encodings = tokenizer(input, return_tensors='pt', padding=True, truncation=True, max_length=256)
  encodings.to(device)
  output = model(**encodings)
  preds = torch.max(output, 1)

  return {'prediction':preds[1], 'confidence':preds[0]}

def evaluate(model, tokenizer, data_loader):

  total_count = 0
  correct_count = 0 

  preds = []
  tot_labels = []

  with torch.no_grad():
    for data in tqdm(data_loader): 

      labels = {}
      labels['label'] = data['label']

      tweets = data['text']

      pred = predict_pcl(tweets, tokenizer, model)

      preds.append(np.array(pred['prediction'].cpu()))
      tot_labels.append(np.array(labels['label'].cpu()))

  # with the saved predictions and labels we can compute accuracy, precision, recall and f1-score
  

  return preds, tot_labels

In [28]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

#your saved model name here
model_name = './models/pcl_bert_finetuned/' 
model = pcl_detection.from_pretrained(model_name)
model.to(device)

###########################
# Validation set
###########################

val_dataset = DpmDataset(tokenizer, validation_set1)
# we don't batch our test set unless it's too big
test_loader = DataLoader(val_dataset)

preds, tot_labels = evaluate(model, tokenizer, test_loader)
tot_labels = np.array(tot_labels)
preds = np.array(preds)
report = classification_report(tot_labels, preds, target_names=["Not PCL","PCL"], output_dict= True)
print(report)

print(report['accuracy'])
print(report['Not PCL']['f1-score'])
print(report['PCL']['f1-score'])

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /home/neal/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /home/neal/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /home/neal/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216b

  0%|          | 0/2393 [00:00<?, ?it/s]

{'Not PCL': {'precision': 0.3964110929853181, 'recall': 0.9931880108991825, 'f1-score': 0.5666537116206762, 'support': 734}, 'PCL': {'precision': 0.9909747292418772, 'recall': 0.3309222423146474, 'f1-score': 0.4961590600994126, 'support': 1659}, 'accuracy': 0.5340576681989135, 'macro avg': {'precision': 0.6936929111135977, 'recall': 0.662055126606915, 'f1-score': 0.5314063858600444, 'support': 2393}, 'weighted avg': {'precision': 0.8086054400599656, 'recall': 0.5340576681989135, 'f1-score': 0.5177817405075227, 'support': 2393}}
0.5340576681989135
0.5666537116206762
0.4961590600994126


## 2.5 Test on provided test set

In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

#your saved model name here
model_name = './models/pcl_bert_finetuned/' 
model = pcl_detection.from_pretrained(model_name)
model.to(device)
###########################
# Test set
###########################

test_dataset = DpmDataset(tokenizer, tedf1)
# we don't batch our test set unless it's too big
test_loader = DataLoader(test_dataset)

preds, tot_labels = evaluate(model, tokenizer, test_loader)
tot_labels = np.array(tot_labels)
preds = np.array(preds)
report = classification_report(tot_labels, preds, target_names=["Not PCL","PCL"], output_dict= True)
print(report)

print(report['accuracy'])
print(report['Not PCL']['f1-score'])
print(report['PCL']['f1-score'])

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /home/neal/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /home/neal/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /home/neal/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216b

  0%|          | 0/2094 [00:00<?, ?it/s]

{'Not PCL': {'precision': 0.9277412834765033, 'recall': 0.9688654353562005, 'f1-score': 0.9478575116159008, 'support': 1895}, 'PCL': {'precision': 0.48695652173913045, 'recall': 0.2814070351758794, 'f1-score': 0.356687898089172, 'support': 199}, 'accuracy': 0.9035339063992359, 'macro avg': {'precision': 0.7073489026078169, 'recall': 0.6251362352660399, 'f1-score': 0.6522727048525364, 'support': 2094}, 'weighted avg': {'precision': 0.8858519961862754, 'recall': 0.9035339063992359, 'f1-score': 0.8916766362138859, 'support': 2094}}
0.9035339063992359
0.9478575116159008
0.356687898089172


## 2.6 Test on Competition Test Set and Upload result

In [32]:
dpm.load_test()

In [33]:
# Add dummy labels
dpm.test_set_df['label'] = 0
dpm.test_set_df

,par_id,art_id,keyword,country,text,label
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to...",0
1,t_1,@@16397324,women,pk,In most poor households with no education chil...,0
2,t_2,@@16257812,migrant,ca,The real question is not whether immigration i...,0
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population...",0
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken C...",0
...,...,...,...,...,...,...
3827,t_3893,@@20319448,migrant,jm,In a letter dated Thursday to European Commiss...,0
3828,t_3894,@@9990672,poor-families,au,They discovered that poor families with health...,0
3829,t_3895,@@37984,migrant,ca,"She married at 19 , to Milan ( Emil ) Badovina...",0
3830,t_3896,@@9691377,immigrant,us,The United Kingdom is n't going to devolve int...,0


In [34]:
def predict_pcl(input, tokenizer, model): 
  model.eval()
  encodings = tokenizer(input, return_tensors='pt', padding=True, truncation=True, max_length=256)
  encodings.to(device)
  output = model(**encodings)
  preds = torch.max(output, 1)

  return {'prediction':preds[1], 'confidence':preds[0]}

def evaluate(model, tokenizer, data_loader):

  total_count = 0
  correct_count = 0 

  preds = []
  tot_labels = []

  with torch.no_grad():
    for data in tqdm(data_loader): 

      tweets = data['text']

      pred = predict_pcl(tweets, tokenizer, model)

      preds.append(pred['prediction'].item())

  return preds

In [35]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

#your saved model name here
model_name = './models/pcl_bert_finetuned/' 
model = pcl_detection.from_pretrained(model_name)
model.to(device)

###########################
# Competition Test set
###########################

test_dataset = DpmDataset(tokenizer, dpm.test_set_df)
# we don't batch our test set unless it's too big
test_loader = DataLoader(test_dataset)

preds = evaluate(model, tokenizer, test_loader)

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /home/neal/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /home/neal/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /home/neal/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216b

  0%|          | 0/3832 [00:00<?, ?it/s]

In [36]:
preds = np.array(preds)
preds = preds.reshape(-1)

In [37]:
Counter(preds)

Counter({0: 3622, 1: 210})

In [38]:
labels2file([[k] for k in preds], 'task1.txt')

In [39]:
# Generate pseduo output for task 2 for upload
# random predictions for task 2
preds_task2 = [[random.choice([0,1]) for k in range(7)] for k in range(0,len(dpm.test_set_df))]
labels2file(preds_task2, 'task2.txt')

In [40]:
!cat task1.txt | head -n 10

0
0
0
0
0
0
0
0
0
0


In [41]:
!cat task2.txt | head -n 10

0,0,1,0,0,0,0
0,1,0,0,0,0,0
0,0,1,0,1,1,0
0,1,1,1,0,0,1
0,0,1,0,1,1,1
0,1,0,1,0,1,1
0,0,0,0,1,0,0
0,1,1,1,1,0,1
1,0,1,0,0,0,0
1,1,1,0,1,0,0


In [42]:
!zip submission.zip task1.txt task2.txt

  adding: res/task1.txt (deflated 96%)
  adding: res/task2.txt (deflated 87%)
